In [10]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime
import os

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 16:39:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/fhvhv_tripdata_2021-06.csv.gz")
)

In [13]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [5]:
df.repartition(12).write.parquet("data/homework/")

In [7]:
os.system("ls -lh data/homework/")

total 284M
-rw-r--r-- 1 user user   0 Apr  3 16:44 _SUCCESS
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00000-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00001-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00002-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00003-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00004-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00005-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00006-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00007-bdf89fad-d395-4734-b089-19c76026d34e-c000.snappy.parquet
-rw-r--r-- 1 user user 24M Apr  3 16:44 part-00008-bdf89fad-d395-4734-b089-1

0

In [11]:
(
    df
    .groupBy(F.dayofmonth("pickup_datetime").alias("day"))
    .count()
    .show()
)

+---+------+
|day| count|
+---+------+
| 28|425909|
| 26|592505|
| 27|509437|
| 12|591339|
| 22|469568|
|  1|417375|
| 13|509039|
|  6|522753|
| 16|479776|
|  3|521408|
| 20|491630|
|  5|604903|
| 19|601189|
| 15|452470|
|  9|483353|
| 17|497133|
|  4|538917|
|  8|462554|
| 23|474599|
|  7|425771|
+---+------+
only showing top 20 rows



In [20]:
(
    df
    .select(F.col(df.dropoff_datetime - df.pickup_datetime).alias("trip_length"))
    .agg({"trip_length": "max"})
    .take(1)
)

[Row(max(trip_length)=datetime.timedelta(days=2, seconds=67964))]

In [21]:
zone_df = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/taxi_zone_lookup.csv")
)

In [22]:
df.printSchema()
zone_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [27]:
f_df = (
    df
    .join(zone_df, df.PULocationID == zone_df.LocationID)
)

In [35]:
(
    f_df
    .groupBy("Zone")
    .count()
    .orderBy(F.col("count").desc())
    .take(10)
)

[Row(Zone='Crown Heights North', count=231279),
 Row(Zone='East Village', count=221244),
 Row(Zone='JFK Airport', count=188867),
 Row(Zone='Bushwick South', count=187929),
 Row(Zone='East New York', count=186780),
 Row(Zone='TriBeCa/Civic Center', count=164344),
 Row(Zone='LaGuardia Airport', count=161596),
 Row(Zone='Union Sq', count=158937),
 Row(Zone='West Village', count=154698),
 Row(Zone='Astoria', count=152493)]